<a href="https://colab.research.google.com/github/okeefe123/MSDS699-Final-Project/blob/main/Final_Notebook.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

*Note: If above banner is not clickable, right click and follow link to access colab notebook

In [1]:
import numpy as np
import pandas as pd
from   sklearn.decomposition   import PCA
from   sklearn.model_selection import *
from   sklearn.model_selection import train_test_split
from   sklearn.pipeline        import Pipeline
from   sklearn.preprocessing   import *
from   sklearn.impute          import *
from   sklearn.linear_model    import *
from   sklearn.compose         import *
from   sklearn.ensemble        import *
import warnings

warnings.filterwarnings("ignore")

# Metrics:

- For this project, I decided to use two metrics:

    1. Mean Absolute Error (MAE): I feel like this metric gives a more interpretable score than RMSE, and a bonus is that it tends to penalize outliers less heavily, which may be more helpful for the small dataset I'm using.
    2. Relative Absolute Error (RAE): While MAE works, it's hard to find context for it at a glance. RAE, on the other hand, scales is against the deviation of the the test set's output values from its average. In the context of my project, this intuition of comparing my model's predictions to worldwide average suicide rates as a benchmark makes it a very intuitive way to see the model's effectiveness at a glance. RAE lower than 1, this model will show better predictions which take into consideration the individual country's attributes rather than a vast generalization of several country suicide rates. 

In [3]:
from   sklearn.metrics         import mean_absolute_error

def relative_absolute_error(test, pred):
    avg_test = [np.mean(test)] * len(test)
    return np.sum(np.abs(pred-test))/np.sum(np.abs(avg_test-test))

# Load in and split the data set

- Because of the small size of the dataset (100 columns representing 100 countries), I decided to split only into train/test and use cross-validation to assess a "success" metric.

In [2]:
df = pd.read_csv("country_stats_mental_health.csv").set_index('Country')
X = df.drop('Suicide Rate 2016', axis=1)
y = df['Suicide Rate 2016']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

# Baseline Model

- Create a baseline regression model to compare with engineered models

In [3]:
categorical_columns = ['Region']
numerical_columns = (X_train.dtypes == float)

con_pipe = Pipeline([('imputer', SimpleImputer(add_indicator=True)),
                    ])
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy="most_frequent")),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))
                    ])

preprocessing = ColumnTransformer([('categorical', cat_pipe, categorical_columns),
                                   ('continuous', con_pipe, numerical_columns)])

pipe = Pipeline([('preprocessing', preprocessing),
                 ('baseline', LinearRegression())])

pipe.fit(X_train, y_train)
y_pred_base = pipe.predict(X_test)

print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_base):.3f}")
print(f"Relative Absolute Error: {relative_absolute_error(y_test, y_pred_base):.3f}")

Mean Absolute Error: 8.816
Relative Absolute Error: 3.009


- First glance tells me there's nowhere to go but up in terms of metrics. The mean absolute error was on the same scale as many country's rates, and the relative absolute error suggests that this performs worse than noise in the true suicide rate values.

# Default Model Selection:
- A simple for loop was used to determine the two best models introduced below. This was done by identifying the best candidates with default hyperparameters from a list of 11 potential candidates. The candidates and their Mean Absolute Error scores were (reporting from another notebook):

    1. BayesianRidge():$\hspace{23mm}$ 3.696
    2. RandomForestRegressor():$\hspace{5mm}$     3.712
    3. ElasticNet(): $\hspace{29mm}$                3.855
    4. Lasso():$\hspace{38mm}$ 3.918
    5. LassoLars():$\hspace{31mm}$ 4.196
    6. Ridge():$\hspace{38mm}$ 6.403
    7. RANSACRegressor():$\hspace{16mm}$ 6.422
    8. SGDRegressor():$\hspace{23mm}$ 7.238
    9. LinearRegression(): $\hspace{17mm}$ 7.986
    10. TheilSenRegressor(): $\hspace{15mm}$ 8.725
    11. HuberRegressor(): $\hspace{19mm}$ 9.238
   
$$ $$

- As can be seen from the list, the appropriate choice of model alone vastly improved the Mean Absolute Error.

# Model 1: Bayesian Ridge Pipeline

In [4]:
# Outline of pipeline creation

# Feature Engineering
categorical_columns = ['Region']
numerical_columns = (X_train.dtypes == float)

con_pipe = Pipeline([('imputer', SimpleImputer(add_indicator=True)),
                     ('quantile', QuantileTransformer(output_distribution='normal')),
                     ('pca', PCA())
                    ])
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy="most_frequent")),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))
                    ])

preprocessing = ColumnTransformer([('categorical', cat_pipe, categorical_columns),
                                   ('continuous', con_pipe, numerical_columns)])


# Creating the pipeline with Bayesian Ridge
pipe = Pipeline([('preprocessing', preprocessing),
                 ('bayesianridge', BayesianRidge(normalize=False))])

### Cross-Validation

- While the imputer hyperparameter was just to check most useful strategy, the other two were more strategic:
    1. PCA: The number of components can only be as large as the number of features, so I decided on multiples of five up to 30, then multiples of 10 from there to the number of features in the data. I expected the number of components to be on the larger side (every feature counts!), and was proven correct.
    2. Quantiles: The number of quantiles is again constrained by the data set. Any larger than 100 will not give any more accurate of a prediction. I predict that each quantile should ideally be given 1-2 points. I wsa actually incorrect about this, with the best parameter being 10 quantiles (i.e. 10 samples per quantile)
    
- From past experience, the number of iterations was by far the most helpful in improving metric scores for these regularized linear regression models. The alpha/lambda hyperparameters also have significance in the regularization approximations, so I thought that it would be useful to test multiple combinations to see how the model reacted. The one trend I see is that the model prefered higher values.

In [5]:
# Define hyperparameters to cross validate
hyperparameters = dict(preprocessing__continuous__imputer__strategy     = ['mean', 'median'],
                       preprocessing__continuous__pca__n_components     = [5, 10, 15, 20, 25, 30, 40, 50, 60],
                       preprocessing__continuous__quantile__n_quantiles = [10, 20, 30, 40, 50, 100],
                       bayesianridge__n_iter                            = [300, 400, 500, 600],
                       bayesianridge__alpha_1                           = [10e-6, 10e-5, 10e-4, 10e-3],
                       bayesianridge__alpha_2                           = [10e-6, 10e-5, 10e-4, 10e-3],
                       bayesianridge__lambda_1                          = [10e-6, 10e-5, 10e-4, 10e-3],
                       bayesianridge__lambda_2                          = [10e-6, 10e-5, 10e-4, 10e-3])

In [6]:
# Randomized search for best parameters (chosen for time efficiency)

cv_rand_br = RandomizedSearchCV(estimator=pipe,
                                 param_distributions=hyperparameters,
                                 n_iter=50,
                                 cv=KFold(n_splits=5),
                                 n_jobs=-1,
                                 verbose=1)

In [7]:
# Find the best model
best_model_br = cv_rand_br.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


In [8]:
# Hyperparameters of best model
best_model_br.best_params_

{'preprocessing__continuous__quantile__n_quantiles': 10,
 'preprocessing__continuous__pca__n_components': 60,
 'preprocessing__continuous__imputer__strategy': 'median',
 'bayesianridge__n_iter': 400,
 'bayesianridge__lambda_2': 0.0001,
 'bayesianridge__lambda_1': 0.001,
 'bayesianridge__alpha_2': 1e-05,
 'bayesianridge__alpha_1': 0.001}

### Model 1 Evaluation:

In [9]:
# Prediction using the above hyperparameters
y_pred_br = best_model_br.predict(X_test)

In [10]:
# Model evaluation with via two metrics
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_br):.3f}")
print(f"Relative Absolute Error: {relative_absolute_error(y_test, y_pred_br):.3f}")

Mean Absolute Error: 3.790
Relative Absolute Error: 1.294


- This is interesting to me, as the default BayesianRidge() I ran in model selection actually performed better than the engineered model above. I must have gotten lucky with the original train/test split! Regardless, it produced much better results than the base model.

# Model 2: Random Forest Regressor

In [11]:
# Outline of pipeline creation

# Feature Engineering
categorical_columns = ['Region']
numerical_columns = (X_train.dtypes == float)

con_pipe = Pipeline([('imputer', SimpleImputer(add_indicator=True)),
                     ('pca', PCA())
                    ])
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy="most_frequent")),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))
                    ])

preprocessing = ColumnTransformer([('categorical', cat_pipe, categorical_columns),
                                   ('continuous', con_pipe, numerical_columns)])

# Creating the pipeline with Random Forest Regressor
pipe = Pipeline([('preprocessing', preprocessing),
                 ('randomforest', RandomForestRegressor(n_jobs=-1))])

### Cross-validation

- I again decided to test both imputer strategies to get a feel for the best way to fill in missing values, and PCA components also have the same logic as before, though it was interesting to see that this model preferred less features than BayesianRidge(). I decided against a scale transformation, as scale doesn't impact Random Forest models (i.e. the loss for a feature split is inherently normalized via the ratio in MSE/MAE), so quantile transformation was tossed from preprocessing.

- As for the model hyperparameters, I took some hints from Terence's lectures in optimizing the trees. 
    1. The number of decision trees (n_estimators) tends to have diminishing returns after a certain amount. In the case of the best model, this value was 50.
    2. Max_features seemed to be the next best hyperparameter to tweak. This establishes how many features are considered for each decision node split, and so can have a big impact on the ultimate random forest aggregate.
    3. Criterion determines how much we would like to penalize outliers during loss calculation. Given the number of features and the limited samples, I hypothesized that mean abolute error would win just because data is so entrenched in the human experience (which can be all over the place). My hypothesis was verified by the best model.
    4. Min samples per split and min samples per leaf are both restrictions which can alter the final state of individual decision trees, so I included it in cross-validation to cover all bases.

In [12]:
# Define hyperparameters to cross validate
hyperparameters = dict(preprocessing__continuous__imputer__strategy     = ['mean', 'median'],
                       preprocessing__continuous__pca__n_components     = [5, 10, 15, 20, 25, 30, 40, 50, 60],
                       randomforest__n_estimators                       = [10, 50, 100, 200, 300, 500],
                       randomforest__min_samples_split                  = [1, 2, 3, 4],
                       randomforest__min_samples_leaf                   = [1, 2, 3, 5, 10, 15],
                       randomforest__max_features                       = ['auto', 'sqrt', 'log2'],
                       randomforest__criterion                          = ["mse", "mae"])

In [13]:
# Randomized search for best parameters (chosen for time efficiency)
cv_random_rf = RandomizedSearchCV(estimator=pipe,
                                 param_distributions=hyperparameters,
                                 n_iter=50,
                                 cv=KFold(n_splits=5),
                                 n_jobs=-1,
                                 verbose=1)

In [14]:
# Find the best model
best_model_rf = cv_random_rf.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


In [15]:
# Hyperparameters of best model
best_model_rf.best_params_

{'randomforest__n_estimators': 50,
 'randomforest__min_samples_split': 4,
 'randomforest__min_samples_leaf': 2,
 'randomforest__max_features': 'log2',
 'randomforest__criterion': 'mae',
 'preprocessing__continuous__pca__n_components': 40,
 'preprocessing__continuous__imputer__strategy': 'median'}

### Model 2 Evaluation:

In [16]:
# Prediction using the above hyperparameters
y_pred_rf = best_model_rf.predict(X_test)

In [17]:
# Model evaluation with via two metrics
print(f"Mean Absolute Error: {mean_absolute_error(y_test, y_pred_rf):.3f}")
print(f"Relative Absolute Error: {relative_absolute_error(y_test, y_pred_rf):.3f}")

Mean Absolute Error: 3.559
Relative Absolute Error: 1.215


- This model performed the best, though in ths specific case the relative absolute error suggests we'd have better luck if using the average of known suicide rates. Still, it performed marginally better than the lucky default model run and much much more accurate than the baseline model outlined above.

# Final Model: Random Forest Regressor

In [35]:
# Final pipeline, see below for final model parameteres
categorical_columns = ['Region']
numerical_columns = (X_train.dtypes == float)

con_pipe = Pipeline([('imputer', SimpleImputer(add_indicator=True,
                                               strategy='median')),
                     ('pca', PCA(n_components=40))
                    ])
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy="most_frequent")),
                     ('ohe', OneHotEncoder(handle_unknown='ignore'))
                    ])

preprocessing = ColumnTransformer([('categorical', cat_pipe, categorical_columns),
                                   ('continuous', con_pipe, numerical_columns)])

# Creating the pipeline with Random Forest Regressor
pipe = Pipeline([('preprocessing', preprocessing),
                 ('randomforest', RandomForestRegressor(n_jobs=-1,
                                                        n_estimators=50,
                                                        min_samples_split=4,
                                                        min_samples_leaf=2,
                                                        max_features='log2',
                                                        criterion='mae'))
                ])

### Final Model HyperParameters:

In [44]:
pipe['randomforest'].get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mae',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 2,
 'min_samples_split': 4,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 50,
 'n_jobs': -1,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

### Best non-default hyperparameters:

- PCA: 
    - n_components = 40


- SimpleImputer:
    - strategy = 'median'


- Random Forest Regressor:
    - n_estimators = 50
    - min_samples_split = 4
    - min_samples_leaf = 2
    - max_features = 'log2'
    - criterion = 'mae'
    

### Parameter Interpretation:

In this model, several aspects were adjusted to create the best final version. The first two were part of the feature engineering for continuous features. Cross validation deemed that imputing missing values with the median of the corresponding feature column was most effective. As for PCA, the number of transformed composite components reduced the number of columns from 66 to 40.

The random forest regressor had the most tweaks to the default hyperparameters. The decision was made to create a random forest out of 50 decision trees, where the minimum number of samples required to split an internal node was 4. In addition, there was a minimum of 2 samples required to be considered a "leaf" node. The maximum features randomly chosen to consider when looking for the best split was log_2(n), where n is the total number of features. Finally, the criteria used to measure the quality of a split was Mean Average Error.

# Conclusion

In conclusion, the Random Forest Regressor proved to be the best model in predicting suicide rates when given the 66 compiled features from the "Mental Health and Suicide Rates" dataset and the "UN Data Country Profiles" dataset. For the run shown in the notebook, the Mean Absolute Error was 3.56, implying that the average prediction for suicide rates for any given country was 3.56 percentage values different than the true value. The Relative Absolute Error was 1.215, meaning that the average absolute error was 1.215x larger than that of suicide rate noise's deviation from the average. While this implies that predicting a country's suicide rate from the average of all countries is still a better prediction method, the random forest regressor model performs much better than the baseline model when comparing metrics.

For this project, many aspects of the pipeline changed as the module progressed. I became aware of much more sophisticated methods of hyperparameter tuning (random cross-validation) which cut out the loops I was using with a primitive grid search. I learned how to include parts of preprocessing into the pipeline, as well as the syntax to analyze the parameters of both the pipeline itself and the cross_validated models. 

While I was hoping for a more accurate model, in the end I was satisfied at just how much better my engineered machine learning models performed compared to my baseline model. The inherent randomness of the random forest regressor made it difficult to determine whether it was the best, but I felt that it most consistently performed the best when dealing with different splits in the full data frame. I found out quickly that the quantile transformer was inconsequential in the outcome of the random forest regressor, as the method of splitting doesn't compare the scales of each feature space and therefore rescaling doesn't affect the decision. 

The question of "why does this matter?" only comes up when we analyze the feature importance found during the model construction. The top ten predictive features all had to do with the way government allocates funds for mental health, education, and infrastructure. Features with "quick fix" mindsets tended to have less importance in the evaluation, instead favoring long term solutions. While it's difficult to say with complete authority, this model does reinforce the importance of education, infrastructure, and mental healthcare accessibility in managing suicide rates within any given country. While it's incredibly difficult to model human mental health in a quantitative way, this project further reinforced my belief that the right to 

Next steps to further improve this pipeline includes introducing deep learning and gradient boosting models, ensembling methods, creating more features from those which already exist in the dataset, and upsampling existing data/finding more data which matches the schema of the current features.

In [ ]:
print(yes)